In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [111]:
df = pd.read_csv('sleep_health_and_lifestyle_dataset.csv')
df["Gender"] = df["Gender"].apply(lambda x: 0 if x == 'Male' else 1)
df[:10]

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,0,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,0,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,0,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,0,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,0,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
5,6,0,28,Software Engineer,5.9,4,30,8,Obese,140/90,85,3000,Insomnia
6,7,0,29,Teacher,6.3,6,40,7,Obese,140/90,82,3500,Insomnia
7,8,0,29,Doctor,7.8,7,75,6,Normal,120/80,70,8000,NaN
8,9,0,29,Doctor,7.8,7,75,6,Normal,120/80,70,8000,NaN
9,10,0,29,Doctor,7.8,7,75,6,Normal,120/80,70,8000,NaN


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(list(df["Occupation"].unique().tolist()))
df["Occupation"] = le.transform(df["Occupation"])

bmi_map = {"Normal":0, "Overweight":1, "Obese":2}
df["BMI Category"] = df["BMI Category"].apply(lambda bmi: "Normal" if "Normal" in bmi else bmi)
df["BMI Category"] = df["BMI Category"].apply(lambda bmi: bmi_map[bmi])

df["Sleep Disorder"] = df["Sleep Disorder"].apply(lambda sd: 1 if sd == "Sleep Apnea" else (2 if sd == "Insomnia" else 0))

df[["Systolic", "Diastolic"]] = df["Blood Pressure"].str.split("/", expand=True)
df = df.drop(columns=["Blood Pressure"])

df.head(10)

   Person ID  Gender  Age  Occupation  Sleep Duration  Quality of Sleep  \
0          1       0   27           9             6.1                 6   
1          2       0   28           1             6.2                 6   
2          3       0   28           1             6.2                 6   
3          4       0   28           6             5.9                 4   
4          5       0   28           6             5.9                 4   
5          6       0   28           9             5.9                 4   
6          7       0   29          10             6.3                 6   
7          8       0   29           1             7.8                 7   
8          9       0   29           1             7.8                 7   
9         10       0   29           1             7.8                 7   

   Physical Activity Level  Stress Level  BMI Category  Heart Rate  \
0                       42             6             1          77   
1                       60        